# Visualize Option Prices

In [14]:
import yfinance as yf

ticker = "TQQQ"

asset = yf.Ticker(ticker)

expirations = asset.options
print("Available Expiration Dates:\n", "\n".join(expirations), sep='')


expiration_date = expirations[-1]

option_chain = asset.option_chain(expiration_date)

calls = option_chain.calls
puts = option_chain.puts


Available Expiration Dates:
2024-12-27
2025-01-03
2025-01-10
2025-01-17
2025-01-24
2025-01-31
2025-02-21
2025-03-21
2025-06-20
2026-01-16
2027-01-15


In [15]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,TQQQ270115C00035000,2024-12-26 20:57:21+00:00,35.0,60.45,56.40,60.50,3.000000,5.221932,5,676,0.711185,True,REGULAR,USD
1,TQQQ270115C00040000,2024-12-26 14:50:12+00:00,40.0,54.25,54.40,57.50,-0.849998,-1.542647,6,781,0.740176,True,REGULAR,USD
2,TQQQ270115C00045000,2024-12-26 16:29:08+00:00,45.0,51.75,51.20,54.00,-1.200001,-2.266290,1,429,0.719180,True,REGULAR,USD
3,TQQQ270115C00050000,2024-12-26 20:13:02+00:00,50.0,49.25,48.20,51.00,-0.450001,-0.905434,4,849,0.707583,True,REGULAR,USD
4,TQQQ270115C00055000,2024-12-20 16:04:33+00:00,55.0,42.00,43.00,48.00,0.000000,0.000000,25,332,0.657047,True,REGULAR,USD
5,TQQQ270115C00056000,2024-12-20 15:43:10+00:00,56.0,40.55,42.50,47.50,0.000000,0.000000,2,96,0.657291,True,REGULAR,USD
6,TQQQ270115C00057000,2024-12-20 16:34:52+00:00,57.0,41.67,42.00,45.20,0.000000,0.000000,5,59,0.629703,True,REGULAR,USD
7,TQQQ270115C00058000,2024-12-20 15:57:06+00:00,58.0,43.50,41.50,46.45,3.889999,9.820750,16,16,0.656376,True,REGULAR,USD
8,TQQQ270115C00059000,2024-12-24 17:29:23+00:00,59.0,42.20,40.50,45.50,0.000000,0.000000,10,91,0.641849,True,REGULAR,USD
9,TQQQ270115C00060000,2024-12-26 17:46:58+00:00,60.0,41.56,40.00,45.00,-1.459999,-3.393768,6,653,0.641483,True,REGULAR,USD


In [16]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,TQQQ270115P00035000,2024-12-24 14:36:42+00:00,35.0,5.05,4.05,6.05,0.000000,0.000000,1.0,558,0.703372,False,REGULAR,USD
1,TQQQ270115P00040000,2024-12-26 15:55:54+00:00,40.0,6.20,5.85,8.00,-0.600000,-8.823535,37.0,417,0.705630,False,REGULAR,USD
2,TQQQ270115P00045000,2024-12-26 16:32:46+00:00,45.0,7.61,5.95,10.00,-1.090000,-12.528732,2.0,403,0.665226,False,REGULAR,USD
3,TQQQ270115P00050000,2024-12-26 20:52:52+00:00,50.0,9.05,8.70,10.90,-0.820000,-8.308001,69.0,540,0.653507,False,REGULAR,USD
4,TQQQ270115P00055000,2024-12-20 16:19:25+00:00,55.0,11.94,10.30,12.95,0.000000,0.000000,2.0,248,0.637516,False,REGULAR,USD
5,TQQQ270115P00056000,2024-12-04 18:41:00+00:00,56.0,11.00,8.50,13.50,0.000000,0.000000,1.0,11,0.602970,False,REGULAR,USD
6,TQQQ270115P00057000,2024-12-11 15:02:49+00:00,57.0,11.22,11.10,12.40,0.000000,0.000000,5.0,4,0.611637,False,REGULAR,USD
7,TQQQ270115P00058000,2024-12-26 14:30:01+00:00,58.0,12.10,11.45,13.95,0.260000,2.195948,1.0,1,0.625797,False,REGULAR,USD
8,TQQQ270115P00059000,2024-12-17 19:38:01+00:00,59.0,12.25,11.70,14.50,0.000000,0.000000,4.0,9,0.622990,False,REGULAR,USD
9,TQQQ270115P00060000,2024-12-26 18:41:30+00:00,60.0,12.79,12.15,14.60,-1.070000,-7.720056,114.0,284,0.616337,False,REGULAR,USD
